# DataParallel 进行多GPU训练
下面将学习如何用DataParalled来使用多GPU，通过PyTorch使用多个GPU非常简单。如下:你可以先将一个模型放在一个GPU下:

In [1]:
import torch
device = torch.device("cuda:0")
# model.to(device)

然后你可以复制所有的张量到GPU:

In [2]:
# mytensor = my_tensor.to(device)

请注意，只是调用my_tensor.to(device)返回一个my_tensor新的复制在GPU上，而不是重写my_tensor。你需要给他分配一个新的张量并且在GPU上使用这个张量.  

在多个GPu中执行前馈，后馈操作是非常自然的，尽管如此，PyTorch默认只会使用一个GPu，通过使用DataParallel让你的模型能够并行运行，你可以很容易在多个GPU上运行你的操作。

In [3]:
# model = torch.nn.DataParallel(model)

## 引入PyTorch模块和定义参数
### 引入模块

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader # 引入数据集和数据加载器

### 参数

In [5]:
input_size = 5 # 输入channel数
output_size = 2 # 输出channel数
batch_size = 30 # batch大小
data_size = 100 # 数据大小

### 设备

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 简单模型
### 实验数据
生成数据，实现一个数据类型，然后使用DataLoader进行加载:

In [7]:
class RandomDataset(Dataset):
    # fixme: 初始化
    def __init__(self, size, length):
        self.len = length
        self.data = torch.rand(length, size) # 随机生成维度为length×size的数据
    
    # fixeme: 实现getitem函数，可以用来逐元素获取数据，这个接口应该是要被DataLoader所调用的
    def __getitem__(self, index):
        return self.data[index]
    
    # fixme: 返回长度
    def __len__(self):
        return self.len

使用DataLoader进行加载数据:

In [8]:
# shuffle设置随机抽取，batch_size设置load的batch大小
data_loader = DataLoader(dataset=RandomDataset(input_size,data_size), batch_size=batch_size, shuffle=True)

### 实验设计
这里仅仅是使用了一个简单的Demo，用来演示可以在多个GPU上并行处理数据，这里使用的模型只是获得一个输入，执行一个线性操作，然后给出一个输出，但是你同样可以使用DataParallel在任何模型上进行使用。  

#### 模型定义

In [20]:
class Model(nn.Module):
    def __init__(self, input_size, output_size):
        super(Model, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
        
    def forward(self, input):
        output = self.fc(input)
        print("模型输入大小:",input.size(),"输出大小",output.size())
        return output

#### 创建模型并且进行数据并行处理
首先需要一个模型的实例，然后验证我们是否有多个GPU，如果我们有多个GPU，我们就可以用nn.DataParallel来包裹我们的模型。然后我们使用mode.to(device)把模型放到GPU中。

In [21]:
model = Model(input_size, output_size)
if torch.cuda.device_count() > 1:
    print("目前有",torch.cuda.device_count(),"个GPUs可以使用.")
    model = nn.DataParallel(model)
model.to(device)

目前有 4 个GPUs可以使用.


DataParallel(
  (module): Model(
    (fc): Linear(in_features=5, out_features=2, bias=True)
  )
)

根据上面的结果可以得到目前你可以使用的GPU给树，并且如果你的GPU个数大于1的时候，就可以将模型使用DataParallel进行包裹，然后再将模型交给设备了.  
运行模型，并且要注意将数据也交给device，然后在对数据进行处理.

In [22]:
step = 1
for data in data_loader:
    input = data.to(device)
    output = model(input)
    print("步数",step,"——输入的维度:",input.size(), ",输出的维度:",output.size())
    step += 1

模型输入大小: torch.Size([8, 5]) 输出大小 torch.Size([8, 2])
模型输入大小: torch.Size([8, 5]) 输出大小 torch.Size([8, 2])
模型输入大小: torch.Size([8, 5]) 输出大小 模型输入大小:torch.Size([8, 2])
 torch.Size([6, 5]) 输出大小 torch.Size([6, 2])
步数 1 ——输入的维度: torch.Size([30, 5]) ,输出的维度: torch.Size([30, 2])
模型输入大小:模型输入大小: torch.Size([8, 5]) 输出大小 torch.Size([8, 2])
模型输入大小: torch.Size([8, 5]) 输出大小 torch.Size([8, 2])
模型输入大小: torch.Size([6, 5]) 输出大小 torch.Size([6, 2])
 torch.Size([8, 5]) 输出大小 torch.Size([8, 2])
步数 2 ——输入的维度: torch.Size([30, 5]) ,输出的维度: torch.Size([30, 2])
模型输入大小: torch.Size([8, 5]) 输出大小 torch.Size([8, 2])
模型输入大小:模型输入大小: torch.Size([8, 5]) 输出大小 torch.Size([8, 2])
 torch.Size([8, 5]) 输出大小 torch.Size([8, 2])
模型输入大小: torch.Size([6, 5]) 输出大小 torch.Size([6, 2])
步数 3 ——输入的维度: torch.Size([30, 5]) ,输出的维度: torch.Size([30, 2])
模型输入大小: torch.Size([3, 5]) 输出大小 torch.Size([3, 2])
模型输入大小: torch.Size([3, 5]) 输出大小 torch.Size([3, 2])
模型输入大小: torch.Size([3, 5]) 输出大小 torch.Size([3, 2])
模型输入大小: torch.Size([1, 5]) 输出大小 torch.Size([1, 2]

这里由于我有4个GPU，因此输出的结果，每一个步数内都有四次model的计算。
## 总结
数据并行自动拆分了你的数据并且将任务单发送到多个GPU上，当每一个模型都完成自己的任务之后，DataParallel会收集和合并这些结果，然后在返回给你。